# 🧠 Premier League Predictor v3.0 (Smart Caching, Betting Odds & Grid Search)

Nesta versão, vamos profissionalizar o pipeline:
1.  **Smart Loading:** Só baixa dados se não tivermos o ficheiro local.
2.  **Betting Insights:** Usar as odds da Bet365 como features (representam o "consenso do mercado").
3.  **Grid Search:** O computador vai testar várias configurações do XGBoost para encontrar a melhor accuracy possível automaticamente.

Imports e Configuração

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import os
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
%matplotlib inline

## 1. Data Acquisition (Recolha de Dados)
Vamos buscar dados reais do `football-data.co.uk`. Vamos carregar várias temporadas consecutivas para que o modelo tenha histórico suficiente para aprender padrões.

* **FTHG**: Full Time Home Goals
* **FTAG**: Full Time Away Goals
* **FTR**: Full Time Result (H=Home, D=Draw, A=Away)

In [ ]:
DATA_FILE = 'premier_league_v3_full.csv'

def get_data(start_year, end_year):
    # 1. Verificar se ficheiro existe localmente
    if os.path.exists(DATA_FILE):
        print(f"Ficheiro local '{DATA_FILE}' encontrado! A carregar...")
        df = pd.read_csv(DATA_FILE)
        df['Date'] = pd.to_datetime(df['Date'])
        # Se quiseres atualizar dados recentes, apaga o ficheiro .csv da pasta e corre isto de novo
        return df
    
    # 2. Se não existe, sacar da net
    print("Ficheiro não encontrado. A fazer download da internet...")
    base_url = "https://www.football-data.co.uk/mmz4281/{}/{}.csv"
    dfs = []
    
    for year in range(start_year, end_year + 1):
        season_str = f"{str(year)[-2:]}{str(year+1)[-2:]}"
        url = base_url.format(season_str, "E0")
        try:
            df = pd.read_csv(url)
            df['Season'] = year
            # Normalizar Data
            df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
            dfs.append(df)
        except Exception as e:
            print(f"Erro no ano {year}: {e}")
            
    full_df = pd.concat(dfs, ignore_index=True)
    full_df = full_df.dropna(subset=['Date', 'FTR'])
    full_df = full_df.sort_values('Date').reset_index(drop=True)
    
    # Guardar para a próxima vez
    full_df.to_csv(DATA_FILE, index=False)
    print("✅ Download concluído e guardado no PC.")
    return full_df

# Carregar dados
df = get_data(2005, 2025)
display(df.tail(3))

## 2. Feature Engineering Completa (ELO + Stats + Odds)

Aqui adicionamos as colunas B365H, B365D, B365A (Odds da Bet365).

In [ ]:
def prepare_features(df, window=5):
    df = df.copy()
    
    # --- 1. ELO SYSTEM ---
    elo_dict = {}
    df['HomeElo'] = 1500.0
    df['AwayElo'] = 1500.0
    k_factor = 20
    
    for i, row in df.iterrows():
        h, a, res = row['HomeTeam'], row['AwayTeam'], row['FTR']
        h_elo = elo_dict.get(h, 1500.0)
        a_elo = elo_dict.get(a, 1500.0)
        
        df.at[i, 'HomeElo'] = h_elo
        df.at[i, 'AwayElo'] = a_elo
        
        if res == 'H': val = 1
        elif res == 'D': val = 0.5
        else: val = 0
        
        exp_h = 1 / (1 + 10**((a_elo - h_elo)/400))
        new_h = h_elo + k_factor * (val - exp_h)
        new_a = a_elo + k_factor * ((1-val) - (1-exp_h))
        
        elo_dict[h] = new_h
        elo_dict[a] = new_a
        
    df['EloDiff'] = df['HomeElo'] - df['AwayElo']
    
    # --- 2. ROLLING STATS ---
    home_stats = df[['Date', 'HomeTeam', 'FTHG', 'FTAG', 'HS', 'HST', 'HC']].copy()
    home_stats.columns = ['Date', 'Team', 'Goals', 'Conceded', 'Shots', 'SoT', 'Corners']
    home_stats['Points'] = df['FTR'].map({'H':3, 'D':1, 'A':0})
    
    away_stats = df[['Date', 'AwayTeam', 'FTAG', 'FTHG', 'AS', 'AST', 'AC']].copy()
    away_stats.columns = ['Date', 'Team', 'Goals', 'Conceded', 'Shots', 'SoT', 'Corners']
    away_stats['Points'] = df['FTR'].map({'A':3, 'D':1, 'H':0})
    
    all_stats = pd.concat([home_stats, away_stats]).sort_values(['Team', 'Date'])
    
    metrics = ['Points', 'Goals', 'Conceded', 'Shots', 'SoT', 'Corners']
    for m in metrics:
        all_stats[f'Avg_{m}'] = all_stats.groupby('Team')[m].transform(
            lambda x: x.shift(1).rolling(window, min_periods=3).mean()
        )
    
    # Merge Home rolling
    df = df.merge(
        all_stats[['Date', 'Team'] + [f'Avg_{m}' for m in metrics]],
        left_on=['Date', 'HomeTeam'],
        right_on=['Date', 'Team'],
        how='left'
    ).drop(columns=['Team'])
    df = df.rename(columns={f'Avg_{m}': f'Home_{m}' for m in metrics})
    
    # Merge Away rolling
    df = df.merge(
        all_stats[['Date', 'Team'] + [f'Avg_{m}' for m in metrics]],
        left_on=['Date', 'AwayTeam'],
        right_on=['Date', 'Team'],
        how='left'
    ).drop(columns=['Team'])
    df = df.rename(columns={f'Avg_{m}': f'Away_{m}' for m in metrics})
    
    # --- 3. BETTING ODDS ---
    if 'B365H' in df.columns:
        df['Prob_Home'] = 1 / df['B365H']
        df['Prob_Draw'] = 1 / df['B365D']
        df['Prob_Away'] = 1 / df['B365A']
        df = df.dropna(subset=['Prob_Home'])
    
    # Preencher só as rolling averages
    rolling_cols = [f for f in df.columns if f.startswith("Home_") or f.startswith("Away_")]
    df[rolling_cols] = df[rolling_cols].fillna(0)
    
    # Remover colunas totalmente vazias
    df = df.dropna(axis=1, how='all')
    
    return df, elo_dict

# Aplicar e verificar colunas
df_processed, elo_tracker = prepare_features(df)
print("Colunas disponíveis para treino:", df_processed.columns.tolist())

## 3. Preparação e Treino do Modelo
Treino Intensivo: Grid Search (Hyperparameter Tuning) Aqui é onde "apertamos" o modelo. Vamos testar várias combinações. Nota: Isto pode demorar 2 ou 3 minutos a correr.

In [ ]:
# Preparar dados
features = ['HomeElo', 'AwayElo', 'EloDiff', 
            'Prob_Home', 'Prob_Draw', 'Prob_Away'] + \
           [c for c in df_processed.columns if 'Home_' in c or 'Away_' in c]

# Filtrar apenas as colunas que realmente existem (caso falte alguma)
features = [f for f in features if f in df_processed.columns]

le = LabelEncoder()
df_processed['Target'] = le.fit_transform(df_processed['FTR'])

# Split
split = int(len(df_processed) * 0.80) # Treinar com 80%, testar com 20% (mais recente)
train = df_processed.iloc[:split]
test = df_processed.iloc[split:]

X_train = train[features]
y_train = train['Target']
X_test = test[features]
y_test = test['Target']

# --- GRID SEARCH ---
print("A iniciar Treino Intensivo (Grid Search)...")

# Definir o modelo base
xgb_model = xgb.XGBClassifier(random_state=42, objective='multi:softprob', eval_metric='mlogloss')

# Definir a grelha de parametros para testar
param_grid = {
    'n_estimators': [100, 200],      # Quantas árvores?
    'max_depth': [3, 4, 5],          # Quão complexa é cada árvore?
    'learning_rate': [0.01, 0.05],   # Quão rápido aprende?
    'subsample': [0.8],              # Evitar overfitting
}

# Configurar a procura
# TimeSeriesSplit garante que a validação cruzada respeita o tempo (não treina no futuro)
tscv = TimeSeriesSplit(n_splits=3)

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=tscv, scoring='accuracy', verbose=1)
grid_search.fit(X_train, y_train)

print(f"\nMelhores Parâmetros encontrados: {grid_search.best_params_}")
best_model = grid_search.best_estimator_

### Matriz de Confusão e accuracy
Vamos ver visualmente onde o modelo erra.
* Eixo Y: O que realmente aconteceu.
* Eixo X: O que o modelo previu.

In [ ]:
# Avaliar o melhor modelo no set de teste (que ele nunca viu)
preds = best_model.predict(X_test)
acc = accuracy_score(y_test, preds)

print(f"\nAccuracy Final (Otimizada): {acc:.2%}")

# Matriz de Confusão
cm = confusion_matrix(y_test, preds)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Matriz de Confusão (XGBoost)')
plt.ylabel('Real')
plt.xlabel('Previsto')
plt.show()

# Ver importância das features
# Vais ver que 'Prob_Home' (Odds) e 'EloDiff' vão estar no topo
importances = pd.Series(best_model.feature_importances_, index=features).sort_values(ascending=False)
plt.figure(figsize=(10,6))
importances.head(15).plot(kind='barh')
plt.title("Top 15 Fatores Mais Importantes")
plt.show()

## 4. Aplicação na "Vida Real"
Aqui está a função final. Ela usa o dicionário `current_elo` (que contém os valores mais recentes após o último jogo do dataset) para fazer previsões sobre jogos futuros.

In [ ]:
def predict_final(home, away, odd_h, odd_d, odd_a):
    # 1. Buscar Stats Recentes
    match_row = df_processed[(df_processed['HomeTeam'] == home) | (df_processed['AwayTeam'] == home)].tail(1)
    if match_row.empty: return "Equipa Home desconhecida"
    # (Simplificação: usar últimas stats conhecidas)
    # Nota: Num sistema de produção real, terias de recalcular as médias com o jogo mais recente
    
    h_elo = elo_tracker.get(home, 1500)
    a_elo = elo_tracker.get(away, 1500)
    
    # Criar input dictionary
    input_data = {
        'HomeElo': h_elo, 'AwayElo': a_elo, 'EloDiff': h_elo - a_elo,
        'Prob_Home': 1/odd_h, 'Prob_Draw': 1/odd_d, 'Prob_Away': 1/odd_a
    }
    
    # Preencher stats de forma (Home_Points, Away_Shots, etc.)
    # Vamos buscar os valores da ultima linha conhecida no dataset
    # (Atenção: Isto assume que o dataset está atualizado até à semana passada)
    
    # Truque para preencher o resto das features com base nas últimas médias conhecidas
    h_row = df_processed[df_processed['HomeTeam'] == home].iloc[-1]
    a_row = df_processed[df_processed['AwayTeam'] == away].iloc[-1]
    
    for feat in features:
        if feat not in input_data:
            if 'Home_' in feat:
                input_data[feat] = h_row[feat] # Usa média histórica
            elif 'Away_' in feat:
                input_data[feat] = a_row[feat] # Usa média histórica

    # Converter para DataFrame
    X_input = pd.DataFrame([input_data])
    X_input = X_input[features] # Garantir ordem
    
    # Prever
    probs = best_model.predict_proba(X_input)[0]
    
    print(f"\n⚔️ {home} vs {away}")
    print(f"   Odds Mercado: {odd_h} | {odd_d} | {odd_a}")
    print(f"🤖 IA Previsão:")
    print(f"   Casa:   {probs[2]*100:.1f}%")
    print(f"   Empate: {probs[1]*100:.1f}%")
    print(f"   Fora:   {probs[0]*100:.1f}%")
    
    winner = np.argmax(probs)
    labels = ['Fora', 'Empate', 'Casa']
    print(f"   >> Aposta Sugerida: {labels[winner]}")


predict_final('Aston Villa', 'Arsenal', 4.05, 3.45, 1.84)